In this project I will develop a classification algorithm based on probabilistic models.

There is a collection of texts in English, Italian, Spanish, German, French, Polish and Portuguese languages in the `data` folder obtained from random Wikipedia articles, see e.g. `Spanish.txt`. (See corresponding `.source.txt` files and links therein for lists of authors.) 

In [1]:
!ls data

English.sources.txt  German.txt		  Portuguese.sources.txt
English.txt	     Italian.sources.txt  Portuguese.txt
French.sources.txt   Italian.txt	  Spanish.sources.txt
French.txt	     Polish.sources.txt   Spanish.txt
German.sources.txt   Polish.txt


In [2]:
with open("data/Spanish.txt") as f:
    print(f.read(500))

tomas zapata sierra medellin colombia  de mayo de  es productor de cine y de teatrocita requerida
sad eyed lady of the lowlands en espanol senorita de ojos tristes de las tierras bajas es una cancion compuesta por el cantante estadounidense bob dylan fue incluida en el album blonde on blonde editado el  de mayo de 
la revista mojo la coloco en el puesto  de su lista de las  mejores canciones de bob dylan
calyptocephalella canqueli es una especie extinta de anfibio anuro perteneciente al genero c


In [3]:
with open("data/German.txt") as f:
    print(f.read(500))

riedhofe ist der name von ortsteilen in deutschland

in badenwurttemberg
riedhofe langenau ortsteil der stadt langenau im albdonaukreis
riedhofe frickingen ortsteil der gemeinde frickingen im bodenseekreis
riedhofe riegel am kaiserstuhl ortsteil der gemeinde riegel am kaiserstuhl im landkreis emmendingen
riedhofe kongen ortsteil der gemeinde kongen im landkreis esslingen
riedhofe leingarten ortsteil der gemeinde leingarten im landkreis heilbronn
riedhofe bad wurzach ortsteil der stadt bad wurzac


These texts are preprocessed: only standard Latin characters kept, diacritics removed, punctuations removed, all letters converted to lowercase. I will use similar preprocessing for new texts. Here are the functions to do it.

### Preprocessing functions

In [4]:
import re
import unicodedata

### FROM: https://stackoverflow.com/a/518232/3025981
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')
### END FROM

def clean_text(s):
    return re.sub("[^a-z \n]", "", strip_accents(s))

### Obtaining character frequencies

First of all, I will find character relative frequencies in texts of each language. I will consider them as probability for character to appear in the multinomial model. The function `get_freqs(text, relative)` takes string `text` as input and returns dictionary which keys are all distinct characters occurred in `text` and values are frequencies (relative if `relative` is `True` and absolute otherwise).

In [5]:
def get_freqs(text, relative=False):
    
    chars_to_freqs_dict = {}
    
    for char in list(text):
        chars_to_freqs_dict[char] = chars_to_freqs_dict.get(char, 0) + 1
    
    if relative:
        text_length = len(list(text))
        chars_to_freqs_dict = {
            char: freq / text_length
            for char, freq in chars_to_freqs_dict.items()
        }
            
    return chars_to_freqs_dict

In [6]:
assert get_freqs('Hello, World!') == {'H': 1, 'e': 1, 'l': 3, 'o': 2, ',': 1,
                                      ' ': 1, 'W': 1, 'r': 1, 'd': 1, '!': 1}

Now let's use the function `get_freqs` to create a dictionary `lang_to_probs` which keys are names of languages (i.e. `'English'`, `'Italian'`, `'Spanish'`, `'German'`, `'French'`, `'Polish'`, `'Portuguese'`) and values are dictionaries of relative frequencies, obtained by processing of corresponding `.txt` files.

In [7]:
from pathlib import Path


directory = Path("data")
ext = ".sources"
paths = Path(directory).glob('**/*.txt',)

lang_to_probs = {
    path.stem: get_freqs(path.read_text(), True)
    for path in paths
    if ext not in path.suffixes
}

### Likelihoods

Now let's start implementing the actual classifier. I begin with a multinomial likelihood function. The function `multinomial_likelihood(probs, freqs)` takes two arguments: `probs` are dictionary of probabilities of each character (in some language) and `freqs` is dictionary of absolute frequencies of each character (in some text we want to classify). This function returns probability to obtain these absolute frequencies from multinomial distribution with given probabilities $P((X_1 = f_1) \cap (X_2 = f_2) \cap \ldots \cap (X_k = f_k))$ provided that $(X_1, \ldots, X_k)$ is a system of multinomially distributed values with probabilities $(p_1, \ldots, p_k)$.

In [8]:
from math import factorial


def prod(list):
    product = 1
    for element in list:
        product = product * element
    return product

def multinomial_likelihood(probs, freqs):
    prob_without_coeff = prod([probs[char] ** freq for char, freq in freqs.items()])
    multinomial_coeff = factorial(sum(freqs.values())) / prod([factorial(freq) for freq in freqs.values()])
    return prob_without_coeff * multinomial_coeff

Let's find likelihood of data with frequencies `{'a': 2, 'b': 1, 'c': 2}` and probabilities `{'a': 0.2, 'b': 0.5, 'c': 0.3}`:

In [9]:
multinomial_likelihood(probs={'a': 0.2, 'b': 0.5, 'c': 0.3}, freqs={'a': 2, 'b': 1, 'c': 2})

0.05400000000000001

Note that the coefficient with factorials depends only on `freqs` (i.e. text that we analyse) and does not depend on `probs`. It means that for all possible languages this coefficient will be the same. As we are going to consider fixed text and compare likelihoods for different languages, we see that we do not need this coefficient in most cases. Let's implement the function `multinomial_likelihood_without_coeff` that returns the same probability as `multinomial_likelihood` but without the coefficient.

In [10]:
def multinomial_likelihood_without_coeff(probs, freqs):
    return prod([probs[char] ** freq for char, freq in freqs.items()])

The corresponding probability becomes smaller:

In [11]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                                     freqs={'a': 2, 'b': 1, 'c': 2})

0.0018000000000000004

Actually, likelihoods become extremely small very quickly when the absolute frequencies in the data are increased. The probability to get the text that coincides with the actual text from a random experiment is extremely small.

In [12]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 3, 'b': 2, 'c': 2})

0.00018000000000000004

In [13]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 3, 'b': 20, 'c': 2})

6.866455078125001e-10

Due to limited precision of computer arithmetic, for frequencies large enough we will get exactly zero likelihood.

In [14]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                                     freqs={'a': 543, 'b': 512, 'c': 2})

0.0

Thus, we usually cannot use likelihoods like this directly. It is a common practice to use _logarithms_ instead of likelihood themself to deal with such a tiny numbers.

### Log likelihood

Let's implement a function `log_likelihood_without_coeff(probs, freqs)` that calculates logarithm of likelihood (without factorial coefficient).

In [15]:
from math import log


def log_likelihood_without_coeff(probs, freqs):
    return sum([freq * log(probs[char]) for char, freq in freqs.items()])

Likelihoods are probabilities, so they are less than 1 and their logarithms are negative. The larger absolute value of log-likelihood, the less is likelihood. 

In [16]:
log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 2, 'b': 1, 'c': 2})

-6.319968614080018

Now we can deal with the inputs that lead to exact zero value previously.

In [17]:
log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 543, 'b': 512, 'c': 2})

-1231.2240885070605

### Maximum likelihood classifier

Now I will use likelihood to choose the best language for some text we want to classify. The function `mle_best(text, lang_to_probs)` takes some `text` and dictionary `lang_to_probs` that I created previously and returns the name of the language such that the likelihood of our data for this language is maximal.

In [18]:
def mle_best(text, lang_to_probs):
    
    cleaned_text = clean_text(text)
    freqs = get_freqs(cleaned_text)
    
    likelihood_dict = {
        language: log_likelihood_without_coeff(probs, freqs)
        for language, probs in lang_to_probs.items()
    }
    
    return max(likelihood_dict, key=likelihood_dict.get)

Let's test it!

In [19]:
# Source: https://en.wikipedia.org/wiki/1134_Kepler
text = """1134 Kepler, provisional designation 1929 SA, is a stony asteroid 
and eccentric Mars-crosser from the asteroid belt, approximately 
4 kilometers in diameter"""
mle_best(text, lang_to_probs)

'English'

In [20]:
# Source: https://pl.wikipedia.org/wiki/(1134)_Kepler
text = """"(1134) Kepler – planetoida z grupy przecinających 
orbitę Marsa okrążająca Słońce w ciągu 4 lat i 145 dni 
w średniej odległości 2,68 au.
"""
mle_best(text, lang_to_probs)

'Polish'

In [21]:
# Source: https://it.wikipedia.org/wiki/1134_Kepler
text = """1134 Kepler è un asteroide areosecante. Scoperto nel 1929, 
presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 
UA e da un'eccentricità di 0,4651458, inclinata di 15,17381° rispetto
"""
mle_best(text, lang_to_probs)

'Italian'

In [22]:
lines = ['Aaa', 'Aa', 'Kepler è un asteroide areosecante. Scoperto nel',
        "presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 "
         "UA e da un'eccentricità di 0,4651458, inclinata",
         "Kepler – planetoida z grupy przecinających orbitę Marsa okrążająca Słońce w ciągu 4 lat i ",
         "Kepler, provisional designation 1929 SA, is a stony asteroid "
         "and eccentric Mars-crosser from the"]

assert mle_best(lines[0], lang_to_probs) == 'Portuguese'
assert mle_best(lines[1], lang_to_probs) == 'Portuguese'
assert mle_best(lines[2], lang_to_probs) == 'French'
assert mle_best(lines[3], lang_to_probs) == 'Italian'
assert mle_best(lines[4], lang_to_probs) == 'Polish'
assert mle_best(lines[5], lang_to_probs) == 'English'

Now let's make the language detection algorithm even better.

### Let's go Bayes

In the Bayesian approach we consider _prior_ probabilities of languages, then use Bayes' rule to find their posterior probabilities and select the language with the largest posterior probability. I begin with finding prior probabilities. Let's create a dictionary `lang_to_prior` whose keys are language names and values are prior probabilities. Also, let's assume that priors are proportional to the number of articles in each language (in thousands). I will use these numbers (in thousands): English: 6090, Italian: 1611, Spanish: 1602, German: 2439, French: 2222, Polish: 1412, Portuguese: 1034

In [23]:
lang_to_articles_count = {
    "English": 6090,
    "Italian": 1611,
    "Spanish": 1602,
    "German": 2439,
    "French": 2222,
    "Polish": 1412,
    "Portuguese": 1034
}

articles_count = sum(lang_to_articles_count.values())

lang_to_prior = {lang: count / articles_count for lang, count in lang_to_articles_count.items()}

Now let's implement a function `bayesian_best(text, lang_to_probs, lang_to_prior)` that takes some text `text`, dictionary `lang_to_probs` created before and `lang_to_prior` with prior probabilities. This function returns the language name with the largest posterior probability.

In [24]:
def bayesian_best(text, lang_to_probs, lang_to_prior):
    
    bayesian_dict = {}
    cleaned_text = clean_text(text)
    freqs = get_freqs(cleaned_text)
    
    bayesian_dict = {
        language: log_likelihood_without_coeff(probs, freqs) + log(lang_to_prior[language])
        for language, probs in lang_to_probs.items()
    }
     
    return max(bayesian_dict, key=bayesian_dict.get)

For example, MLE algorithm believes that word `"The"` belongs go German language.

In [25]:
mle_best("The", lang_to_probs)

'German'

However, if we take into account that English is more popular in Wikipedia, the results changes:

In [26]:
bayesian_best("The", lang_to_probs, lang_to_prior)

'English'

In [27]:
lines = ['a', 'Kepler è un asteroide areosecante. Scoperto nel',
        "presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 "
         "UA e da un'eccentricità di 0,4651458, inclinata",
         "Kepler – planetoida z grupy przecinających orbitę Marsa okrążająca Słońce w ciągu 4 lat i ",
         "Kepler, provisional designation 1929 SA, is a stony asteroid "
         "and eccentric Mars-crosser from the"]

answers = ['English', 'French', 'Italian', 'Polish', 'English']

for line, answer in zip(lines, answers):
    assert bayesian_best(line, lang_to_probs, lang_to_prior) == answer

### Measuring uncertainty

Finally, let's get posterior probability of how certain the Bayesian algorithm is in its classification. Let's implement function `bayesian_posterior(text, lang_to_probs, lang_to_prior, test_lang)` that takes some `text`, `lang_to_probs`, `lang_to_prior` and a particular language `test_lang` we are interested in and returns posterior probability for this language provided this text.

In [28]:
def bayesian_posterior(text, lang_to_probs, lang_to_prior, test_lang):
    
    bayesian_posterior_dict = {}
    cleaned_text = clean_text(text)
    freqs = get_freqs(cleaned_text)
    
    bayesian_denominator = sum({
        language: multinomial_likelihood(probs, freqs) * lang_to_prior[language]
        for language, probs in lang_to_probs.items()
    }.values())
    
    bayesian_posterior_dict = {
        language: multinomial_likelihood(probs, freqs) * lang_to_prior[language] / bayesian_denominator
        for language, probs in lang_to_probs.items()
    } 
    
    return bayesian_posterior_dict[test_lang]

In [29]:
bayesian_posterior("The", lang_to_probs, lang_to_prior, "German")

0.26863814640453393

In [30]:
bayesian_posterior("The", lang_to_probs, lang_to_prior, "English")

0.5346266046898721

In [31]:
bayesian_posterior("Das", lang_to_probs, lang_to_prior, "English")

0.36596793151737517

In [32]:
bayesian_posterior("Das", lang_to_probs, lang_to_prior, "German")

0.12095519926831602

One can see that the algorithm believes that `"Das"` belongs to English. This is due to the small amount of data (only three letters) and high prior for English. However, it is not very certain: the posterior is only 0.37. On the other hand, `"The"` belongs to `"English"` with much larger posterior probability.

## Conclusions

So, we constructed the classifier algorithm. Actually, it is a version of a well-known naive Bayesian classifier. Of course, this classifier is far from perfect: for example, it completely ignores words and deals only with characters and their frequencies. Nevertheless, it works rather well despite being so simple. Also it shows several important concepts: likelihood, maximum likelihood estimation, Bayesian estimations and so on.